In [32]:
import torch
import numpy as np
from transformers import BertTokenizer
from typing import List


class DatasetForBert(torch.utils.data.Dataset):

    def __init__(self, text_list: List[str], label_list: List[str], config_labels: dict, config_tokenizer: dict):
        bert_tokenizer = BertTokenizer.from_pretrained(config_tokenizer["bert_tokenizer_name"])
        self.labels = [config_labels[label] for label in label_list]
        self.texts = [bert_tokenizer(text,
                                     padding=config_tokenizer["padding"],
                                     max_length=config_tokenizer["max_length"],
                                     truncation=config_tokenizer["truncation"],
                                     return_tensors=config_tokenizer["return_tensors"]) for text in text_list]

    def __len__(self):
        return len(self.labels)

    def get_labels(self, idx):
        return np.array(self.labels[idx])

    def get_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):
        texts = self.get_texts(idx)
        y = self.get_labels(idx)

        return texts, y

In [33]:
from torch import nn
from transformers import BertModel


class BertClassifier(nn.Module):

    def __init__(self, pretrained_name: str, layers: list, num_classes: int, activation_func=nn.ReLU(), dropout=0.3):
        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained(pretrained_name)
        self.model_layers = [nn.Linear(layers[0], layers[1])]
        self.activation_func = activation_func
        self.dropout = nn.Dropout(dropout)

        for layer_idx in range(2, len(layers) - 1):
            self.model_layers.append(self.activation_func)
            self.model_layers.append(nn.Linear(layers[layer_idx - 1], layers[layer_idx]))
        self.model_layers.append(self.dropout)
        self.model_layers.append(nn.Linear(layers[-2], num_classes))

        self.fc_module = nn.Sequential(*self.model_layers)

    def forward(self, input_id, mask):
        _, pooled = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        out = self.fc_module(pooled)
        return out

In [34]:
config = {
    "model_name": "bert",
    "bert_settings": {
        "layers": [768, 256, 50],
        "dropout": 0.5,
        "model_name": 'bert-base-cased',
        "num_classes": 5,
        "activation_func": nn.ReLU()
    },
    "train_settings": {
        "batch_size": 50
    },
    "tokenizer_config": {
        "padding": "max_length",
        "max_length": 512,
        "truncation": True,
        "return_tensors": "pt",
        "bert_tokenizer_name": 'bert-base-cased',
    },
    "labels": {'business': 0,
               'entertainment': 1,
               'sport': 2,
               'tech': 3,
               'politics': 4
               },

}

In [47]:
from typing import Tuple
from dataset import DatasetForBert
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm

tokenizer_config = config["tokenizer_config"]
config_labels = config["labels"]

np.random.seed(0)


def evaluation():
    pass


def load_df(path: str) -> pd.DataFrame:
    return pd.read_csv(path)


def generate_datasets(df: pd.DataFrame, val_size: float, test_size: float, batch_size):
    train, validation = train_test_split(df, test_size=(val_size + test_size))
    validation, test = train_test_split(validation, test_size=test_size / (val_size + test_size))
    print(len(train), len(validation), len(test))
    test_dataloader, train_dataloader, val_dataloader = create_dataloader(batch_size, test, train, validation)
    return train_dataloader, val_dataloader, test_dataloader


def create_dataloader(batch_size, test, train, validation):
    print("Start to create DataLoader")
    train_data = DatasetForBert(train.Text, train.Category, config_labels, tokenizer_config)
    val_data = DatasetForBert(validation.Text, validation.Category, config_labels, tokenizer_config)
    test_data = DatasetForBert(test.Text, test.Category, config_labels, tokenizer_config)
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=test.shape[0])

    return test_dataloader, train_dataloader, val_dataloader


def train_model(model, train_dataloader, val_dataloader, learning_rate=0.01, epochs=5):
    print("Start Train")
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)
    model.to(device)
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):
            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)
            output = model(input_id, mask)

            batch_loss = criterion(output, train_label.long())
            print(batch_loss.item())
            total_loss_train += batch_loss.item()

            acc = (output.argmax(dim=1) == train_label).sum().item()
            print("Accuracy", acc, f"For {train_label.shape} labels")
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:
                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()

                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataloader): .3f} \
                | Train Accuracy: {total_acc_train / len(train_dataloader): .3f} \
                | Val Loss: {total_loss_val / len(val_dataloader): .3f} \
                | Val Accuracy: {total_acc_val / len(val_dataloader): .3f}')


In [57]:
def wrapper_model_bootstrap(config: dict):
    if config['model_name'] == "bert":
        bert_config = config["bert_settings"]
        bert = BertModel.from_pretrained('bert-base-cased')
        model = BertClassifier(pretrained_name=bert_config['model_name'],
                               layers=bert_config["layers"],
                               num_classes=bert_config["num_classes"],
                               activation_func=bert_config["activation_func"],
                               dropout=bert_config["dropout"])
        return model
    return NoSuchModelWrapper
model = wrapper_model_bootstrap(config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_re

In [58]:
for param in model.bert.encoder.layer[:-1].parameters():
    param.requires_grad = False

In [54]:
df = load_df("dataset/BBC_News_Train.csv")
train_settings = config["train_settings"]

train_dataloader, val_dataloader, test_dataloader = generate_datasets(df=df,
                                                                      val_size=0.15,
                                                                      test_size=0.15,
                                                                      batch_size=16)

1043 223 224
Start to create DataLoader


In [55]:
import warnings
warnings.filterwarnings("ignore")

In [59]:
%%timeit
train_model(model, train_dataloader, val_dataloader, learning_rate=0.001)

Start Train








  0%|          | 0/66 [00:00<?, ?it/s]

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:79] data. DefaultCPUAllocator: not enough memory: you tried to allocate 100663296 bytes.

In [53]:
del model